In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from functions import *
import csv
import numpy as np


# 1 Folder Data loading
The code is loading multiple CSV files from a specified folder path and converting them into pandas DataFrames.

In [ ]:
folder_path = "data/e production=ep sanyo ep sanyo 002" 
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

dataframes = []
dfarrnames = []

# Loop through the CSV files and load only those ending with "Format01=Kreis 5-064" and "TBA_CU"
for csv_file in csv_files:
    if "Format01=Kreis" in csv_file:
        if "TBA_Zyk" in csv_file:
            dfarr = pd.read_csv(csv_file, skiprows=[1])
            dfarr['Zeit'] = dfarr['Zeit'].apply(convert_to_linear_time)
            dfarr['Zeit'] = dfarr['Zeit'] - dfarr['Zeit'].iloc[0]
            dfarr['Zeit'] = dfarr['Zeit'] / 3600
            dataframes.append(dfarr)
            dfarrnames.append(csv_file)

noFiles = 0
for i, dfarr in enumerate(dataframes):
    num_rows, num_columns = dfarr.shape
    print(f"DataFrame {i + 1} - Rows: {num_rows}, Columns: {num_columns}")
    noFiles += 1
print(noFiles)

test that data is loaded succesfully

In [ ]:
num_rows, num_columns = dataframes[1].shape

print(f"Number of rows (length): {num_rows}")
print(f"Number of columns (width): {num_columns}")
# dataframes[0].head()

Deleting Invalid data

In [ ]:
valid_dataframes = []

for i,df in enumerate(dataframes):
    # Check if the 15th column is named 'Spannung'
    if len(df.columns) > 14 and df.columns[15] == 'Spannung':
        valid_dataframes.append(df)
    else:
        print('deleted : ',i)

# Update dataframes with valid dataframes
dataframes = valid_dataframes

noFiles = 0
for i, dfarr in enumerate(dataframes):
    num_rows, num_columns = dfarr.shape
    print(f"DataFrame {i + 1} - Rows: {num_rows}, Columns: {num_columns}")
    noFiles += 1
print(noFiles)

### Crop The Values


This script iterates through a list of dataframes and performs the following operations:

1. Seprate the data farame of each cycle or `Zyklus`.
2. Extracts a subset of the filtered dataframe from `3.65-3.85`.
3. Filters the subset based on the `Spannung` column values.
4. Calculates the maximum and minimum values of `AhAkku` column in the filtered dataframe.
5. Interpolates 50 values for `AhAkku` column.
6. Writes the interpolated values along with the difference between maximum and minimum `AhAkku` values to a CSV file.

CSV File Format:
- Each row corresponds to a set of interpolated values along with the difference between max and min AhAkku values.
- The columns represent the interpolated values (Interpolated_1 to 'Interpolated_50') followed by `Max_Min_Ahakku_Difference`.

Note: Ensure that the 'dataframes' variable is a list containing pandas DataFrames.

    

In [30]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # writer.writerow(['Interpolated_1', 'Interpolated_2', ..., 'Interpolated_50', 'Max_Min_Ahakku_Difference'])

    I = 0
    for i, df in enumerate(dataframes):
        for zyclus_no in range(1, df['Zyklus'].max() + 1):
            filtered_df = df[(df['Zyklus'] == zyclus_no)]
            quarter_len = len(filtered_df) // 4
            filtered_df = filtered_df.iloc[quarter_len:]

            filtered_df_ahakku = filtered_df[(filtered_df['Spannung'] >= 3.65) & (filtered_df['Spannung'] <= 3.85)]
            max_ahakku = filtered_df['AhAkku'].max()
            min_ahakku = filtered_df['AhAkku'].min()

            ahakku_values = filtered_df_ahakku['AhAkku'].values
            if len(ahakku_values) > 0:
                # Interpolate 50 values
                interpolated_values = np.interp(np.linspace(0, len(ahakku_values) - 1, 50), np.arange(len(ahakku_values)), ahakku_values)

                # Write interpolated values along with max_ahakku - min_ahakku to the CSV file
                writer.writerow(list(interpolated_values) + [max_ahakku - min_ahakku])

        I += 1